# Information Retrieval - Programming Assignment 3
**Retrieval + Re-Ranking using PyTerrier**  
*Author: Febi Imanuela & Luthfi Balaka*

---

For Programming Assignment 3, you are asked to perform re-ranking with learning-to-rank model on the given dataset. In order to do so, you need to preprocess the dataset, index the document collection, ideate the re-ranking features, train the model, and evaluate the performance for further analysis.

PyTerrier is a helpful tool to carry out IR operations. Therefore, we provide a PyTerrier template for this assignment. More information and example can be referred from the previous [Tutorial 1](https://colab.research.google.com/drive/10NpHgtotDQv6bQXEv3CmqyRVQ0BXknlC?usp=sharing) and PyTerrier documentation.


---


*   Fill in the `✋TODO` section
*   Provide justification for your choices
*   Report your experiment results and analysis


---


Happy exploring!

# Setup

In [27]:
!pip install python-terrier;
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git;
!pip install datasets;
!pip install gdown;
!pip install rank_bm25;
!pip install sentence-transformers;
!pip install xgboost;


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/terrierteam/pyterrier_t5.git; to c:\users\notal\appdata\local\temp\pip-req-build-worho1wu


  Running command git clone --filter=blob:none --quiet 'https://github.com/terrierteam/pyterrier_t5.git;' 'C:\Users\notal\AppData\Local\Temp\pip-req-build-worho1wu'
  remote: Repository not found.
  fatal: repository 'https://github.com/terrierteam/pyterrier_t5.git;/' not found
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet 'https://github.com/terrierteam/pyterrier_t5.git;' 'C:\Users\notal\AppData\Local\Temp\pip-req-build-worho1wu' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet 'https://github.com/terrierteam/pyterrier_t5.git;' 'C:\Users\notal\AppData\Local\Temp\pip-req-build-worho1wu' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release o


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import re
import gdown
import pyterrier as pt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from pyterrier.measures import *
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import xgboost as xgb
from sklearn.model_selection import ParameterGrid

if not pt.started():
  pt.init(version='snapshot')

d:\IR\TP3\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\notal\AppData\Local\Temp\ipykernel_10708\4056902049.py:13: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: jitpack 2024-09-24 17:43), helper_version=0.0.8]
C:\Users\notal\AppData\Local\Temp\ipykernel_10708\4056902049.py:14: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
The following code will have the same effect:
pt.terrier.set_version('snapshot')
pt.java.init() # optional, forces java initialisation
  pt.init(version='snapshot')


# Data Preparation

In [29]:
# Load dataset
url = "https://drive.google.com/drive/folders/12AiGXUUlduwVJJrSOYp4RvLWNS8_X_xH?usp=drive_link"
gdown.download_folder(url, output=".", quiet=True)

['.\\corpus.csv',
 '.\\dev.csv',
 '.\\queries.csv',
 '.\\test.csv',
 '.\\train.csv']

In [2]:
collections = pd.read_csv("corpus.csv")
queries = pd.read_csv("queries.csv")

train_qrels = pd.read_csv("train.csv")
dev_qrels = pd.read_csv("dev.csv")
test_qrels = pd.read_csv("test.csv")

In [3]:
collections.head()

,id,text
0,49,"Colorâurine can be a variety of colors, most..."
1,12913,Rio de Janeiro: Annual Weather Averages. Febru...
2,14964,The judiciary (also known as the judicial syst...
3,17272,Painless swelling of the feet and ankles is a ...
4,18352,"Later that day, the National Hurricane Center ..."


In [4]:
queries.head()

,id,text
0,597651,what color is amber urine
1,88585,causes of swollen ankles and feet
2,508811,symptoms of strep throat for an adult
3,412886,is ilovemakonnen ovo
4,532152,uneven chest color


In [5]:
train_qrels.head()

,query-id,corpus-id,score
0,597651,49,1
1,88585,17272,1
2,508811,28092,1
3,412886,56458,1
4,532152,73237,1


## ✋ TODO

In [6]:
def remove_nonalphanum(text):
  pattern = re.compile('[\W_]+')
  return pattern.sub(' ', text)

In [ ]:
# 1. Change ID document from corpus
collections.rename(columns = {
    "id": "docno"
}, inplace = True)

## 2. Remove Nonalphanumeric on the text

collections["text"] = collections["text"].apply(    ).apply(lambda x: x.lower())
collections["docno"] = collections["docno"].apply(lambda x: f"D{x}")

tokenized_corpus = [doc.split(" ") for doc in collections["text"]]

collections.head()

,docno,text
0,D49,colorâ urine can be a variety of colors most o...
1,D12913,rio de janeiro annual weather averages februar...
2,D14964,the judiciary also known as the judicial syste...
3,D17272,painless swelling of the feet and ankles is a ...
4,D18352,later that day the national hurricane center n...


In [8]:
# 3. Change ID from query
queries.rename(columns = {
    "id": "qid",
    "text": "query"
}, inplace = True)

queries["query"] = queries["query"].apply(remove_nonalphanum).apply(lambda x: x.lower())
queries["qid"] = queries["qid"].apply(lambda x: f"Q{x}")

sampled_queries = queries.sample(n=50, random_state=42)

sampled_queries.head()

,qid,query
611,Q750731,what is george soros real name
4443,Q495130,set default browser to windows
1419,Q570901,what are the largest countries in africa
5584,Q259417,how long does respite last
3094,Q36505,average fixed cost formula


In [9]:
train_qrels.rename(columns = {
    "query-id": "qid",
    "corpus-id": "docno",
    "score": "label"
}, inplace = True)

train_qrels["qid"] = train_qrels["qid"].apply(lambda x: f"Q{x}")
train_qrels["docno"] = train_qrels["docno"].apply(lambda x: f"D{x}")
train_qrels["label"] = train_qrels["label"].apply(lambda x: 1 if x >= 1 else 0)

train_qrels.head()

,qid,docno,label
0,Q597651,D49,1
1,Q88585,D17272,1
2,Q508811,D28092,1
3,Q412886,D56458,1
4,Q532152,D73237,1


In [10]:
dev_qrels.rename(columns = {
    "query-id": "qid",
    "corpus-id": "docno",
    "score": "label"
}, inplace = True)

dev_qrels["qid"] = dev_qrels["qid"].apply(lambda x: f"Q{x}")
dev_qrels["docno"] = dev_qrels["docno"].apply(lambda x: f"D{x}")
dev_qrels["label"] = dev_qrels["label"].apply(lambda x: 1 if x >= 1 else 0)

dev_qrels.head()

,qid,docno,label
0,Q174249,D7067348,1
1,Q87892,D7069601,1
2,Q264827,D7071066,1
3,Q206117,D7072160,1
4,Q196232,D7072326,1


In [ ]:
test_qrels.rename(columns = {
    "query-id": "qid",
    "corpus-id": "docno",
    "score": "label"
}, inplace = True)

test_qrels["qid"] = test_qrels["qid"].apply(lambda x: f"Q{x}")
test_qrels["docno"] = test_qrels["docno"].apply(lambda x: f"D{x}")
test_qrels["label"] = test_qrels["label"].apply(lambda x: 1 if x >= 1 else 0)

# Also included all the test qrels on the queries (So that test_queries will work)
test_qrels_qids = test_qrels['qid'].unique()
queries_in_test_qrels = queries[queries['qid'].isin(test_qrels_qids)]

# include also dev

np.random.seed(42)

dev_qrels_qids = np.random.choice(dev_qrels['qid'].unique(), size=30, replace=False)
queries_in_dev_qrels = queries[queries['qid'].isin(dev_qrels_qids)]

sampled_queries = pd.concat([sampled_queries, queries_in_test_qrels, queries_in_dev_qrels]).drop_duplicates()

print(len(sampled_queries))

test_qrels.head(100)

122


,qid,docno,label
0,Q19335,D1720387,0
1,Q19335,D1871222,0
2,Q19335,D1958102,0
3,Q19335,D2046505,1
4,Q19335,D2186129,0
...,...,...,...
95,Q130510,D617865,0
96,Q130510,D6229732,0
97,Q130510,D6486232,1
98,Q130510,D6862766,0


In [12]:
# Creating training, test, and dev queries

train_queries = sampled_queries.merge(train_qrels[['qid']].drop_duplicates(), on='qid', how='inner')

train_queries.head()

,qid,query
0,Q750731,what is george soros real name
1,Q495130,set default browser to windows
2,Q570901,what are the largest countries in africa
3,Q36505,average fixed cost formula
4,Q204149,holiday inn in des moines


In [13]:
test_queries = sampled_queries.merge(test_qrels[['qid']].drop_duplicates(), on='qid', how='inner')

test_queries.head()

,qid,query
0,Q19335,anthropological definition of environment
1,Q47923,axon terminals or synaptic knob definition
2,Q87181,causes of left ventricular hypertrophy
3,Q87452,causes of military suicide
4,Q104861,cost of interior concrete flooring


In [14]:
dev_queries = sampled_queries.merge(dev_qrels[['qid']].drop_duplicates(), on='qid', how='inner')
print(len(dev_queries))
dev_queries.head()

43


,qid,query
0,Q259417,how long does respite last
1,Q925766,what xbox services are down
2,Q686739,what is a heritage oak
3,Q735387,what is county for seattle
4,Q1039195,who is the actress that played gretchen schwar...


# Indexing

## ✋ TODO

In [15]:
# TODO: adjust indexer

!rm -rf ./index
import os

index_path = os.path.abspath("./index/")
pd_indexer = pt.DFIndexer(index_path, \
                          type = pt.index.IndexingType(1), \
                          tokeniser = pt.index.TerrierTokeniser('utf'), \
                          stemmer = None, \
                          stopwords = None, \
                          blocks = True, \
                          verbose = True)
index_ref = pd_indexer.index(collections["text"], collections)

C:\Users\notal\AppData\Local\Temp\ipykernel_10708\652436866.py:7: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  pd_indexer = pt.DFIndexer(index_path, \
100%|██████████| 6992/6992 [00:07<00:00, 889.22documents/s] 


In [16]:
# Get Index Info

print("Number of corpus", len(collections))
print("Number of queries", len(queries))
print("Number of train data", len(train_qrels))
print("Number of test data", len(test_qrels))
print("Number of dev data", len(dev_qrels))

print("="*80)

index_fact = pt.IndexFactory.of(index_ref)
print(index_fact.getCollectionStatistics().toString())

Number of corpus 6992
Number of queries 6577
Number of train data 5535
Number of test data 1036
Number of dev data 1005
Number of documents: 6992
Number of terms: 30149
Number of postings: 282234
Number of fields: 0
Number of tokens: 407801
Field names: []
Positions:   true



# Baseline

In [17]:
bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")

C:\Users\notal\AppData\Local\Temp\ipykernel_10708\3225921946.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")


In [18]:
pt.Experiment(
    [bm25], # models
    test_queries, # queries
    test_qrels, # qrels
    eval_metrics=[P@10, "map", "recip_rank", nDCG@10],
    names=["BM25"], # for model's name readability
)

,name,P@10,map,recip_rank,nDCG@10
0,BM25,0.437209,0.492925,0.684582,0.586706


# Re-rank

## Feature

### ✋ TODO

In [28]:
# TODO: propose and implement features
from sklearn.metrics.pairwise import cosine_similarity

# Example:
def jaccard(doc, query):
    doc_set = set(doc.split())
    query_set = set(query.split())
    intersection = len(doc_set & query_set)
    union = len(doc_set | query_set)
    return intersection / union if union > 0 else 0

def bm25_score(doc, query, bm25_model):
    return bm25_model.get_scores([query])[0]

def tfidf_cosine(doc, query, tfidf_vectorizer):
    vectors = tfidf_vectorizer.transform([doc, query])
    return cosine_similarity(vectors[0], vectors[1])[0, 0]

def query_length_normalized_overlap(doc, query):
    doc_terms = set(doc.split())
    query_terms = set(query.split())
    overlap = len(doc_terms & query_terms)
    return overlap / len(query_terms) if len(query_terms) > 0 else 0

def embedding_cosine(doc, query, embedding_model):
    doc_embedding = embedding_model.encode(doc)
    query_embedding = embedding_model.encode(query)
    return cosine_similarity([doc_embedding], [query_embedding])[0, 0]

def generate_features(doc, query, bm25_model, tfidf_vectorizer, embedding_model):
    return np.array([
        # jaccard(doc, query),
        bm25_score(doc, query, bm25_model),
        tfidf_cosine(doc, query, tfidf_vectorizer),
        query_length_normalized_overlap(doc, query),
        embedding_cosine(doc, query, embedding_model)
    ])

In [29]:
bm25_model = BM25Okapi(tokenized_corpus)

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(collections)

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

features = pt.apply.doc_features(
    lambda row: generate_features(
        row["text"], row["query"], bm25_model, tfidf_vectorizer, embedding_model
    )
)

In [30]:
K = 30

In [31]:
# Get "text" column using `pt.text.get_text(index_ref, "text")`
pipeline = (bm25 % K) >> pt.text.get_text(index_ref, "text") >> (features ** bm25)

## Learning-to-Rank

### ✋ TODO

In [32]:
# TODO: perform hyperparameter tuning on the LambdaMART

# Below hyperparameters are taken from the example on PyTerrier documentation:
# https://pyterrier.readthedocs.io/en/latest/ltr.html#gradient-boosted-trees-lambdamart

# XGBoost hyperparameters documentation:
# https://xgboost.readthedocs.io/en/stable/parameter.html
param_grid = {
    "learning_rate": [0.05, 0.1, 0.2],
    "gamma": [0, 1, 2, 5],
    "min_child_weight": [0.1, 1, 3],
    "max_depth": [3, 6],
    "n_estimators": [100, 300],
}

def evaluate_model(params):
    model = xgb.sklearn.XGBRanker(
        objective="rank:ndcg",
        random_state=42,
        **params,
    )
    
    lmart_pipe = pipeline >> pt.ltr.apply_learned_model(model, form="ltr")
    
    lmart_pipe.fit(train_queries, train_qrels, dev_queries, dev_qrels)
    
    res = pt.Experiment(
        [lmart_pipe],
        dev_queries,
        dev_qrels,
        eval_metrics=[P@10, "map", "recip_rank", nDCG@10],
        names=["L2R"]
    )
    
    print(res)
    return res.loc[0, "nDCG@10"]

best_score = -float("inf")
best_params = None

print(list(ParameterGrid(param_grid)))

for params in ParameterGrid(param_grid):
    score = evaluate_model(params)
    print(f"Score: {score}")
    if score > best_score:
        best_score = score
        best_params = params

print(f"Best Params: {best_params}")
print(f"Best Score: {best_score}")

final_model = xgb.sklearn.XGBRanker(
    objective="rank:ndcg",
    random_state=42,
    **best_params
)

[{'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 0.1, 'n_estimators': 100}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 0.1, 'n_estimators': 300}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 300}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 300}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 0.1, 'n_estimators': 100}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 0.1, 'n_estimators': 300}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100}, {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 300}, {'gamma': 0, 'learnin

In [33]:
lmart_pipe = pipeline >> pt.ltr.apply_learned_model(final_model, form = "ltr")
lmart_pipe.fit(train_queries, train_qrels, test_queries, test_qrels)

# Evaluation


## ✋ TODO

In [34]:
eval_results = pt.Experiment([bm25, lmart_pipe], \
                            test_queries, \
                            test_qrels, \
                            eval_metrics=[P@10, "map", "recip_rank", nDCG@10], \
                            names=["BM25", "L2R"], \
                            baseline=0) # statistical significance test

In [35]:
eval_results

,name,map,recip_rank,P@10,nDCG@10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.492925,0.684582,0.437209,0.586706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L2R,0.550545,0.828488,0.486047,0.684375,24.0,14.0,0.017163,16.0,4.0,0.006937,18.0,8.0,0.019295,26.0,10.0,0.002906
